In [ ]:
!pip install transformers
!pip install pytorch-pretrained-bert
!pip install spacy ftfy==4.4.3
!python -m spacy download en
!pip install accelerate -U
!pip install wandb

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import torch
from pytorch_pretrained_bert import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel

import argparse
import logging
from tqdm import trange

import torch
import torch.nn.functional as F
import numpy as np


from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments

In [ ]:
from google.colab import drive
import pandas as pd

# Google Drive 마운트
drive.mount('/content/drive')

# 데이터셋 경로
dataset_path = '/content/drive/MyDrive/aiffelthon_mine/GiTi/MIntRec'
train_file_path = dataset_path + "/train.tsv"
test_file_path = dataset_path + "/test.tsv"
dev_file_path = dataset_path + "/dev.tsv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 데이터 불러오기
train = pd.read_csv(train_file_path, sep='\t')
# train.head()

train.tail()  #1334개

,season,episode,clip,text,label
1329,S05,E15,113,"speaking of cereal, you sound just like tony t...",Joke
1330,S06,E04,396,this job is hard enough.,Complain
1331,S05,E11,151,and i need a floor supervisor,Ask for help
1332,S06,E03,458,glenn gave me ultimate executive power.,Flaunt
1333,S04,E11,85,it lowers your blood pressure and increases yo...,Introduce


In [ ]:
test = pd.read_csv(test_file_path, sep='\t')
# test.head()

test.tail()  #445개

,season,episode,clip,text,label
440,S05,E18,366,"i mean, i do kind of have to go.",Leave
441,S05,E18,395,"oh, okay, well, yeah. don't--do not give him t...",Prevent
442,S05,E19,389,"guys, i sent carol to lunch so that we could t...",Inform
443,S05,E08,90,"i'm sorry, garrett",Apologise
444,S05,E09,540,your fault because the two of you don't know,Criticize


In [ ]:
dev = pd.read_csv(dev_file_path, sep='\t')
# dev.head()

dev.tail()  #445개

,season,episode,clip,text,label
440,S04,E05,134,thank you. happy 0th birthday.,Thank
441,S05,E10,115,hey.,Greet
442,S04,E16,418,cool shoes.,Praise
443,S05,E13,52,"jonah, i'm not taking you off the schedule.",Inform
444,S04,E14,507,"yes, except i already told corporate that i ga...",Inform


In [ ]:
label_to_index = {
    'Complain' : 0,
    'Praise' : 0,
    'Apologise' : 0,
    'Apologise': 0,
    'Thank' : 0,
    'Criticize': 0,
    'Care': 0,
    'Agree' : 0,
    'Taunt' : 0,
    'Flaunt' : 0,
    'Oppose' : 0,
    'Joke' : 0,
    'Inform' : 1,
    'Advise': 1,
    'Arrange': 1,
    'Introduce': 1,
    'Comfort' : 1,
    'Leave' : 1,
    'Prevent' : 1,
    'Greet' : 1,
    'Ask for help' : 1,
}

# index_to_label = {v : k for k,v in label_to_index.items()}
# index_to_label  # {1: 'Ask for help', 0: 'Flaunt'}

In [ ]:
# train_test_split으로 나누지 말고, train은 train data, test는 test data로
# dev로 (객관적인) 성능평가할 수 있게 코드를 짜기

train_texts = train['text'].tolist()
train_labels = train['label'].tolist()

test_texts = test['text'].tolist()
test_labels = test['label'].tolist()

dev_texts = dev['text'].tolist()
dev_labels = dev['label'].tolist()

In [ ]:
# 라벨 개수
num_labels = len(set(label_to_index.values()))
print('Label 개수 :', num_labels)

Label 개수 : 2


In [ ]:
train_labels = [label_to_index[label] for label in train_labels]
test_labels = [label_to_index[label] for label in test_labels]
dev_labels = [label_to_index[label] for label in dev_labels]

In [ ]:
len(train_labels)  # 1067
len(test_labels)   # 267

type(train_labels)  # list

type(train_labels[0]) # int

int

In [ ]:
# GPT-2 tokenizer 및 model 초기화
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=num_labels) # 바이너리 분류를 위해 num_labels=2로 설정합니다.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
# 토크나이징 및 데이터셋 형식 변경
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
dev_encodings = tokenizer(dev_texts, truncation=True, padding=True)

In [ ]:
class MIntRecDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = MIntRecDataset(train_encodings, train_labels)
test_dataset = MIntRecDataset(test_encodings, test_labels)
dev_dataset = MIntRecDataset(dev_encodings, dev_labels)

In [ ]:
%cd /content/drive/MyDrive/aiffelthon_mine/GiTi

/content/drive/MyDrive/aiffelthon_mine/GiTi


In [ ]:
# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir = './results',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=100,
    report_to = "wandb"

)

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset
    # dev_dataset=dev_dataset
)

In [ ]:
# 모델 학습
trainer.train()

wandb: Currently logged in as: yoojinshin9918 (5fingers). Use `wandb login --relogin` to force relogin


Step,Training Loss
100,0.822900
200,0.619200
300,0.703600
400,0.507300
500,0.378800
600,0.430800
700,0.308600
800,0.311400
900,0.223700
1000,0.218800


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1002, training_loss=0.45161552120189735, metrics={'train_runtime': 136.7216, 'train_samples_per_second': 29.271, 'train_steps_per_second': 7.329, 'total_flos': 65356849741824.0, 'train_loss': 0.45161552120189735, 'epoch': 3.0})

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# 검증 데이터셋에 대한 예측
predictions = trainer.predict(dev_dataset)
# 57% - test dataset
# 88% - dev dataset

# 예측된 라벨
predicted_labels = predictions.predictions.argmax(axis=1)

# 실제 라벨
true_labels = dev_labels

# 정확도 계산
accuracy = accuracy_score(true_labels, predicted_labels)
print("Accuracy:", accuracy)

# 분류 보고서 출력
print("Classification Report:")
print(classification_report(true_labels, predicted_labels))

Accuracy: 0.8808988764044944
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.89      0.89       249
           1       0.86      0.87      0.87       196

    accuracy                           0.88       445
   macro avg       0.88      0.88      0.88       445
weighted avg       0.88      0.88      0.88       445



In [ ]:
import wandb

In [ ]:
# WandB 프로젝트 초기화
wandb.init(project='gpt2_mintrec', name='3rd_try')

# 검증 데이터셋에 대한 평가
eval_results = trainer.evaluate()

# 평가 결과 WandB에 기록
wandb.log(eval_results)

train/epoch,▁▂▃▃▄▅▆▆▇██
train/global_step,▁▂▃▃▄▅▆▆▇██
train/learning_rate,█▇▆▆▅▄▃▃▂▁
train/loss,█▆▇▄▃▃▂▂▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,3.0
train/global_step,1002
